In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, Dataset, DataLoader, TensorDataset

print(torch.__version__)
print(f"MPS available: {torch.backends.mps.is_available()}")
print(f"CUDNN available: {torch.backends.cudnn.is_available()}")
import glob

from nilearn import image

import nibabel as nib
import numpy as np

2.6.0+cu118
MPS available: False
CUDNN available: True


In [13]:
# torch.nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1, 18, 9, padding="same")
        self.conv2 = nn.Conv1d(18, 18, 9, padding="same")
        self.conv3 = nn.Conv1d(18, 1, 1, padding="same")
        self.bn = nn.BatchNorm1d(18)
        self.dropout1 = nn.Dropout(0.10)
        self.dropout2 = nn.Dropout(0.5)

    def forward(self, x):
        x = self.conv1(x)
        x = F.sigmoid(x)
        
        x = self.conv2(x)
        x = self.bn(x)
        x = F.sigmoid(x)

        x = self.dropout1(x)

        x = self.conv2(x)
        x = self.bn(x)
        x = F.sigmoid(x)

        x = self.dropout1(x)

        x = self.conv2(x)
        x = self.bn(x)
        x = F.sigmoid(x)
        
        x = self.dropout1(x)

        x = self.conv2(x)
        x = self.bn(x)
        x = F.sigmoid(x)

        x = self.dropout1(x)

        x = self.conv2(x)
        x = self.bn(x)
        x = F.sigmoid(x)
        
        x = self.dropout1(x)

        x = self.conv2(x)
        x = self.bn(x)
        x = F.sigmoid(x)

        x = self.dropout1(x)
        
        output = self.conv3(x)
        
        return output

In [14]:
# load model
model_saved = Net()
model_saved.load_state_dict(torch.load("test_bay2_50epochs.pt", weights_only=True))
model_saved.eval()
model_saved.to("cuda")


Net(
  (conv1): Conv1d(1, 18, kernel_size=(9,), stride=(1,), padding=same)
  (conv2): Conv1d(18, 18, kernel_size=(9,), stride=(1,), padding=same)
  (conv3): Conv1d(18, 1, kernel_size=(1,), stride=(1,), padding=same)
  (bn): BatchNorm1d(18, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
)

In [15]:
root_path = '/shared/datasets/private/bipolar'

BD_key = '/derivatives/fmriprep/'

BD_path = root_path + BD_key

bold_key = '_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'

file_ls = glob.glob(BD_path + '**/*' + bold_key, recursive=True)

file_ls = [filename for filename in file_ls if 'task-rest_' in filename]
file_ls = [filename for filename in file_ls if 'run-1' in filename]
print(len(file_ls))

def apply_bm_ts(img, bm_path = '/shared/home/zeming/utils/MNI152_T1_2mm_brain_mask.nii.gz'):
    bm_mask_raw = nib.load(bm_path)

    BM_THR = 0

    # Binarize
    bm_mask = image.resample_to_img(bm_mask_raw, img, interpolation = 'nearest').get_fdata()
    bm_mask = bm_mask>BM_THR
    affine = img.affine
    header = img.header
    # apply the gm mask to img
    img = img.get_fdata()
    img_masked = img[bm_mask,:]
    img = np.zeros_like(img)  # Same shape as the original 4D image
    # Place the masked data back into the 4D array at the corresponding voxel locations
    img[bm_mask, :] = img_masked

    img = nib.Nifti1Image(img, affine = affine, header = header)
    return img

61


In [36]:
file_ls[0]

'/shared/datasets/private/bipolar/derivatives/fmriprep/sub-016/ses-bhb/func/sub-016_ses-bhb_task-rest_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'

In [16]:
nifti_path = file_ls[0]

# load data
img = image.load_img(nifti_path)

# apply brain mask
img = apply_bm_ts(img)

In [17]:
data = img.get_fdata()

X, Y, Z, T = data.shape

# Calculate mean and standard deviation for each voxel across the time dimension
voxel_means = np.mean(data, axis=3, keepdims=True)
voxel_stds = np.std(data, axis=3, keepdims=True)

# Normalize the data for each voxel
data = (data - voxel_means) / (voxel_stds + 1e-8)  # Add a small value to avoid division by zero

# Reshape the data
data_2d = data.reshape(-1, T)
mean_np = voxel_means.reshape(-1, 1)
std_np = voxel_stds.reshape(-1, 1)

In [18]:
mask = ~np.all(data_2d == 0, axis=1)  # same as before

valid_data = data_2d[mask]
valid_means = mean_np[mask]
valid_stds = std_np[mask]

data_input = torch.from_numpy(valid_data).float().to("cuda")
data_input = data_input.unsqueeze(1)  # Add channel dimension



In [19]:
batch_size = 12

outputs = []

with torch.no_grad():
    for i in range(0, data_input.size(0), batch_size):
        batch = data_input[i:i+batch_size]
        out = model_saved(batch)  # shape: [B, 1, T] or whatever your model returns
        outputs.append(out.cpu())

outputs = torch.cat(outputs, dim=0).squeeze(1).numpy()

In [21]:
output_denorm = outputs * valid_stds + valid_means
result_denorm = np.zeros_like(data_2d)
result_denorm[mask] = output_denorm

result_denorm_4d = result_denorm.reshape(X, Y, Z, T)

new_img_denorm = nib.Nifti1Image(result_denorm_4d, affine=img.affine, header=img.header)

In [35]:
new_img_denorm.to_filename('test_bay2_50epochs.nii.gz')